## Detección del rostro y facciones con OpenCV y Haarcascades

Vamos a utilizar OpenCV y Haarcascades para la detección de rostros en python. Lo haremos en tiempo real, utilizando nuestra propia webcam.

OpenCV es una librería muy potente de visión por computador o visión artificial que nació en los laboratorios de Intel. Para la detección de los rostros haremos uso de Haarcascades preentrenados para la tarea. Estos clasificadores preenterandos fundcionan mediante una combinación de clasificadores especializados en las diferentes características del rostro. Para la detección de facciones haremo suso de dlib.

_Notas importantes_: 
* Si en vez de pulsar "q" cierras la ventana de la webcam clickando en la X, tendrás que reiniciar el Kernel. Esto es debido a cómo funcionan tanto Jupyter como la captura de video de la webcam.
* Siempre ejecuta tanto el bloque de inicialización del entorno y luego, el de procesado de imagen u opencv dará error.
* Hay una linea para reducir el tamaño de la imagen que captura la webcam. Esta comentada por defecto. Esto es otra técnica de optimización que en mi caso, no hace falta porque mi webcam tiene la resolución de una GameBoy, pero puede que en vuestro caso sí lo sea.

In [ ]:
%matplotlib inline

import cv2
import numpy as np
import dlib  
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.feature import canny
from skimage import exposure

# cargamos los datos necesarios de los clasificadores preentrenados
cascade_path = "./haarcascades/haarcascade_frontalface_default.xml"
predictor_path = "./pred/shape_predictor_68_face_landmarks.dat"

# cargamos los clasificadores de detección de rostro y facciones faciales
faceCascade = cv2.CascadeClassifier(cascade_path)
predictor = dlib.shape_predictor(predictor_path)

# inicializamos la captura de webcam
cap = cv2.VideoCapture(0)
draw_points = True

En realidad, este notebook hace dos cosas: detectar rostros e identificar las facciones de la cara.

Detectar las facciones es algo costoso para hacerlo en tiempo real, pero con una pequeña técnica de optimización se puede conseguir. Lo que hacemos una vez detectado el rostro es establecer un área de trabajo para detectar las facciones de dicho rostro y no tener que buscarlos por toda la imagen capturada. Estamos indicando al detector de facciones dónde está la cara exactamente para tener que procesar menos datos.

La forma en la que funciona, a grosso modo es la siguiente:

* Tendremos un bucle infinito que seguirá capturando desde la webcam hasta que pulsemos la letra "q". ¡Acuérdate de esto para cerrar la captura o tendrás que reiniciar el Kernel!
* Para cada frame que capturamos de la webcam:
    * Convertimos a blanco y negro.
    * Ecualizamos (si es necesario...)
    * Suavizamos (si es necesario...)
    * Detectamos los rostros en la imagen en blanco y negro y filtrada
    
    * Para cada rostro detectado en la imagen:
        * Establecemos un area rectangular donde buscar las facciones.
        * Le damos ese area a dlib para que encuentre dichas facciones.
        * Pintamos las facciones sobre el frame capturado en color.
    
   

In [ ]:
# Bucle infinito de captura ¡Pulsa "q" en tu teclado para cancelarlo y apagar la webcam!
while(True):
    # Capturamos el frame
    ret, frame = cap.read()
    # Redimensionamos a tamaño 800 x 600 píxeles.
    # frame = cv2.resize(frame, (800, 600))

    # Operamos sobre nuestro frame capturado: aquí puedes probar varias cosas!   
    # Convertimos a blanco y negro
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Ecualizamos la imagen, la técnica que ya vimos en el worksheet
    gray = cv2.equalizeHist(gray)
    # suavizamos la imagen, esto puede no ser necesario e incluso empeorar los resultados
    gray = cv2.bilateralFilter(gray,9,75,75)
    
    # filtered = sobel(gray)
    # filtered = exposure.equalize_hist(gray)
    # filtered = exposure.equalize_adapthist(gray, clip_limit=0.5)
    # cv2.imshow('frame', filtered)
    # Buscamos rostros en la imagen capturada
    faces = faceCascade.detectMultiScale(
            frame,
            scaleFactor = 1.3,
            minNeighbors = 4,
            minSize = (100,100),
            flags = cv2.CASCADE_SCALE_IMAGE
            )
    
    # Para cada rostro detectado en la imagen...
    for (x, y, w, h) in faces:
        # Dibujamos un rectángulo indicando el área de dicha imagen
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Establecemos un área de trabajo para detectar las facciones en el rostro actual
        dlib_rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h)) 
        detected_landmarks = predictor(gray, dlib_rect).parts()
        
        # Pintamos las facciones
        landmarks = np.matrix([[p.x, p.y] for p in detected_landmarks])
        if draw_points:
            for idx, point in enumerate(landmarks):
                pos = (point[0, 0], point[0, 1])  

                # Anotamos las facciones
                cv2.putText(frame, str(idx), pos,  
                       fontFace=cv2.FONT_HERSHEY_SIMPLEX,  
                       fontScale=0.4,  
                       color=(0, 0, 255))  

                # Dibujamos los puntos detectados de las facciones
                cv2.circle(frame, pos, 1, color=(0, 255, 255))

    # Pintamos el frame resultante
    #cv2.imshow('frame', frame)
    
    # Si pulsamos q, se cierra el bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Se libera y cierra la captura de webcam
cap.release()
cv2.destroyAllWindows()

Si quisiésemos simplemente aplicar algún filtro de los vistos en el worksheet con scikit-image a la imagen capturada de la webcam, podríamos hacerlo con el siguiente código:

In [4]:
%matplotlib inline

import cv2
import numpy as np
import dlib  
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.feature import canny
from skimage import exposure

# inicializamos la captura de webcam
cap = cv2.VideoCapture(0)
draw_points = True

In [5]:
# Bucle infinito de captura ¡Pulsa "q" en tu teclado para cancelarlo y apagar la webcam!
while(True):
    # Capturamos el frame
    ret, frame = cap.read()
    # Redimensionamos a tamaño 800 x 600 píxeles.
    # frame = cv2.resize(frame, (800, 600))

    # Operamos sobre nuestro frame capturado: aquí puedes probar varias cosas!
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    filtered = sobel(gray)
    #filtered = exposure.equalize_hist(gray)
    #filtered = exposure.equalize_adapthist(gray, clip_limit=0.5)
    #filtered = exposure.equalize_adapthist(filtered, clip_limit=0.5)
    
    # Pintamos el frame resultante
    cv2.imshow('frame', filtered)
    
    # Si pulsamos q, se cierra el bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Se libera y cierra la captura de webcam
cap.release()
cv2.destroyAllWindows()